随着引擎和 SQL 执行的停止，我们准备开始一些 Alchemy。 SQLAlchemy Core 和 ORM 的核心元素是 SQL 表达式语言，它允许流畅、可组合地构建 SQL 查询。这些查询的基础是表示表和列等数据库概念的 Python 对象。这些对象统称为数据库元数据。

SQLAlchemy 中数据库元数据最常见的基础对象称为MetaData 、 Table和Column 。下面的部分将说明如何在面向 Core 的风格和面向 ORM 的风格中使用这些对象。

[文档链接]{https://docs.sqlalchemy.org/en/20/tutorial/metadata.html}

In [1]:
from sqlalchemy import MetaData
metadata_obj = MetaData()

In [2]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

In [3]:
# 查看name字段
user_table.c.name

Column('name', String(length=30), table=<user_account>)

In [4]:
# 查看表所有字段
user_table.c.keys()

['id', 'name', 'fullname']

In [5]:
# 查看表主键
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [6]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

当在一个对象中使用ForeignKey对象时 Column定义，我们可以省略其数据类型 Column ;它是从相关列的数据类型自动推断出来的，在上面的示例中是user_account.id列的Integer数据类型。